In [289]:
import pandas as pd
import csv
import numpy as np

import time
import matplotlib.pyplot as plt
import collections
import sys
import os

import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn import tree
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC


from keras.models import Sequential

from pprint import pprint

In [290]:
NCAA_pd = pd.read_csv('NCAA_Final.csv')

NCAA_pd.head()



,team_id,season,team_name,team_mascot,seed,total_wins,total_losses,total_games_played,adj_win_perc,str_of_sched,...,average_field_goals_att,average_three_points_made,average_three_points_att,average_free_throws_made,average_free_throws_att,average_two_points_made,average_two_points_att,average_points_in_paint,average_efficiency,average_true_shooting_att
0,9b166a3f-e64b-4825-bb6b-92c6f0418263,2017,Arizona,Wildcats,S4,27,8,35,0.7707,0.5515,...,57.23,6.77,18.34,16.09,21.17,22.06,38.89,33.60,93.97,66.54
1,77a69fb0-1355-4342-ac09-b4cc7949d95e,2017,Murray State,Racers,E12,24,6,32,0.8056,0.4869,...,56.34,8.47,22.47,15.75,21.47,18.72,33.88,32.88,90.63,65.79
2,61a3908a-7492-4b6f-809e-12c61976bb0a,2017,Loyola (IL),Ramblers,S11,31,6,38,0.8596,0.5352,...,51.66,7.29,18.42,11.95,16.47,18.89,33.24,32.95,83.11,58.91
3,6ed15092-2670-450a-99c2-61d861e87644,2017,Texas,Longhorns,S10,19,15,34,0.5274,0.5699,...,59.12,7.21,22.53,12.97,19.50,18.74,36.59,32.82,76.74,67.70
4,b2fda957-e15c-4fb2-8a13-6e58496f561e,2017,Oklahoma,Sooners,M10,18,14,32,0.5484,0.5597,...,64.44,9.38,26.13,15.94,21.38,20.44,38.31,36.94,93.44,73.84


In [291]:
pprint(NCAA_pd.columns)

Index(['team_id', 'season', 'team_name', 'team_mascot', 'seed', 'total_wins',
       'total_losses', 'total_games_played', 'adj_win_perc', 'str_of_sched',
       'rpi', 'total_minutes_played', 'total_field_goals_made',
       'total_field_goals_att', 'total_field_goals_pct',
       'total_two_points_made', 'total_two_points_att', 'total_two_points_pct',
       'total_three_points_made', 'total_three_points_att',
       'total_three_points_pct', 'total_blocked_att', 'total_free_throws_made',
       'total_free_throws_att', 'total_free_throws_pct',
       'total_offensive_rebounds', 'total_defensive_rebounds',
       'total_rebounds', 'total_assists', 'total_turnovers',
       'total_assists_turnover_ratio', 'total_steals', 'total_blocks',
       'total_personal_fouls', 'total_tech_fouls', 'total_points',
       'total_fast_break_pts', 'total_flagrant_fouls',
       'total_points_off_turnovers', 'total_second_chance_pts',
       'total_ejections', 'total_foulouts', 'total_points_in_paint

In [292]:
NCAA_Seasons_pd = NCAA_pd.loc[:, ["team_name", "season", "seed", "total_wins", "adj_win_perc", "str_of_sched", "rpi", "total_field_goals_pct", 
                              "total_two_points_pct", "total_three_points_pct", "total_free_throws_pct", "total_true_shooting_pct", 
                              "average_rebounds", "average_blocks", "average_turnovers", "average_efficiency", "average_fast_break_pts", 
                              "average_points_off_turnovers", "average_second_chance_pts", "average_minutes", "average_points"]]


pprint (NCAA_Seasons_pd)


                team_name  season seed  total_wins  adj_win_perc  \
0                 Arizona    2017   S4          27        0.7707   
1            Murray State    2017  E12          24        0.8056   
2             Loyola (IL)    2017  S11          31        0.8596   
3                   Texas    2017  S10          19        0.5274   
4                Oklahoma    2017  M10          18        0.5484   
5            Kansas State    2017   S9          25        0.6527   
6           West Virginia    2017   E5          26        0.6871   
7                     TCU    2017   M6          21        0.6090   
8              Texas Tech    2017   E3          27        0.7315   
9                  Kansas    2017   M1          31        0.7705   
10          Georgia State    2017  S15          22        0.6918   
11                Montana    2017  W14          24        0.8000   
12                Houston    2017   W6          27        0.7931   
13             Cincinnati    2017   S2          

In [293]:
teamList = NCAA_Seasons_pd['team_name'].tolist()
pprint (teamList)

['Arizona',
 'Murray State',
 'Loyola (IL)',
 'Texas',
 'Oklahoma',
 'Kansas State',
 'West Virginia',
 'TCU',
 'Texas Tech',
 'Kansas',
 'Georgia State',
 'Montana',
 'Houston',
 'Cincinnati',
 'Wichita State',
 'UNC Greensboro',
 'South Dakota State',
 'Cal State Fullerton',
 'Marshall',
 'New Mexico State',
 'Stephen F. Austin',
 'Texas Southern',
 'Virginia Tech',
 'North Carolina State',
 'Virginia',
 'Miami (FL)',
 'Syracuse',
 'Florida State',
 'Clemson',
 'North Carolina',
 'Duke',
 'Lipscomb',
 'Bucknell',
 'Radford',
 'Wright State',
 'Nevada',
 'San Diego State',
 'Auburn',
 'Kentucky',
 'Missouri',
 'Florida',
 'Tennessee',
 'Alabama',
 'Arkansas',
 'Texas A&M',
 'Buffalo',
 'Purdue',
 'Ohio State',
 'Michigan State',
 'Michigan',
 'UMBC',
 'Pennsylvania',
 'Charleston',
 'Gonzaga',
 'Xavier',
 'Villanova',
 'Providence',
 'Creighton',
 'Seton Hall',
 'Butler',
 'St. Bonaventure',
 'Davidson',
 'Rhode Island',
 'Iona',
 'Arizona',
 'Murray State',
 'Loyola (IL)',
 'Texas',


In [294]:
E_2017 = ["Villanova", "Butler", "St. Bonaventure", "Murray State", "Marshall", "Stephen F. Austin", "Cal State Fullerton", 
         "Radford", "Purdue", "Texas Tech", "Wichita State", "West Virginia", "Florida", "Arkansas", "Virginia Tech", "Alabama"]

S_2017 = ["Virginia", "Texas", "Loyola (IL)", "Davidson", "Buffalo", "Wright State", "Georgia State", "UMBC", "Cincinnati", "Tennessee", 
          "Arizona", "Kentucky", "Miami (FL)", "Nevada", "Creighton", "Kansas State"]

M_2017 = ["Kansas", "Oklahoma", "Syracuse", "New Mexico State", "Charleston", "Bucknell", "Iona", "Pennsylvania", "Duke", 
          "Michigan State", "Auburn", "Clemson", "TCU", "Rhode Island", "Seton Hall", "North Carolina State"]

W_2017 = ["Xavier", "Providence", "San Diego State", "South Dakota State", "UNC Greensboro", "Montana", "Lipscomb", "Texas Southern", 
          "North Carolina", "Michigan", "Gonzaga", "Ohio State", "Houston", "Texas A&M", "Missouri", "Florida State"]

E_2018 = ["Villanova", "Butler", "St. Bonaventure", "Murray State", "Marshall", "Stephen F. Austin", "Cal State Fullerton", 
         "Radford", "Purdue", "Texas Tech", "Wichita State", "West Virginia", "Florida", "Arkansas", "Virginia Tech", "Alabama"]

S_2018 = ["Virginia", "Texas", "Loyola (IL)", "Davidson", "Buffalo", "Wright State", "Georgia State", "UMBC", "Cincinnati", "Tennessee", 
          "Arizona", "Kentucky", "Miami (FL)", "Nevada", "Creighton", "Kansas State"]

M_2018 = ["Kansas", "Oklahoma", "Syracuse", "New Mexico State", "Charleston", "Bucknell", "Iona", "Pennsylvania", "Duke", 
          "Michigan State", "Auburn", "Clemson", "TCU", "Rhode Island", "Seton Hall", "North Carolina State"]

W_2018 = ["Xavier", "Providence", "San Diego State", "South Dakota State", "UNC Greensboro", "Montana", "Lipscomb", "Texas Southern", 
          "North Carolina", "Michigan", "Gonzaga", "Ohio State", "Houston", "Texas A&M", "Missouri", "Florida State"]

In [295]:
def getTeamName(team_name):
    return NCAA_Seasons_pd[NCAA_Seasons_pd['team_name'] == team_name].values[0][0]

In [296]:
getTeamName('Kansas')

'Kansas'

In [297]:
NCAA_Seasons_pd.dtypes

team_name                        object
season                            int64
seed                             object
total_wins                        int64
adj_win_perc                    float64
str_of_sched                    float64
rpi                             float64
total_field_goals_pct             int64
total_two_points_pct            float64
total_three_points_pct          float64
total_free_throws_pct           float64
total_true_shooting_pct         float64
average_rebounds                float64
average_blocks                  float64
average_turnovers               float64
average_efficiency              float64
average_fast_break_pts          float64
average_points_off_turnovers    float64
average_second_chance_pts       float64
average_minutes                 float64
average_points                  float64
dtype: object

In [301]:
##### STUCK HERE HELP ###############
def getSeasonData(team_name, year):
    season = NCAA_Seasons_pd[NCAA_Seasons_pd.season == year]
    total_wins = NCAA_Seasons_pd[NCAA_Seasons_pd.total_wins == team_name]
    adj_win_perc = NCAA_Seasons_pd[NCAA_Seasons_pd.adj_win_perc == team_name]
    str_of_sched = NCAA_Seasons_pd[NCAA_Seasons_pd.str_of_sched == team_name]
    rpi = NCAA_Seasons_pd[NCAA_Seasons_pd.rpi == team_name]
    total_field_goals_pct = NCAA_Seasons_pd[NCAA_Seasons_pd.total_field_goals_pct == team_name]
    total_two_points_pct = NCAA_Seasons_pd[NCAA_Seasons_pd.total_two_points_pct == team_name]
    total_three_points_pct = NCAA_Seasons_pd[NCAA_Seasons_pd.total_three_points_pct == team_name]
    total_free_throws_pct = NCAA_Seasons_pd[NCAA_Seasons_pd.total_free_throws_pct == team_name]
    total_true_shooting_pct = NCAA_Seasons_pd[NCAA_Seasons_pd.total_true_shooting_pct == team_name]
    average_rebounds = NCAA_Seasons_pd[NCAA_Seasons_pd.average_rebounds == team_name]
    average_blocks = NCAA_Seasons_pd[NCAA_Seasons_pd.average_blocks == team_name]
    average_turnovers = NCAA_Seasons_pd[NCAA_Seasons_pd.average_turnovers == team_name]
    average_efficiency = NCAA_Seasons_pd[NCAA_Seasons_pd.average_efficiency == team_name]
    average_fast_break_pts = NCAA_Seasons_pd[NCAA_Seasons_pd.average_fast_break_pts == team_name]
    average_points_off_turnovers = NCAA_Seasons_pd[NCAA_Seasons_pd.average_points_off_turnovers == team_name]
    average_second_chance_pts = NCAA_Seasons_pd[NCAA_Seasons_pd.average_second_chance_pts == team_name]
    average_minutes = NCAA_Seasons_pd[NCAA_Seasons_pd.average_minutes == team_name]
    average_points = NCAA_Seasons_pd[NCAA_Seasons_pd.average_points == team_name]
    
    
    return [total_wins, adj_win_perc, str_of_sched, rpi, total_field_goals_pct,
            total_two_points_pct, total_three_points_pct, total_free_throws_pct, total_true_shooting_pct,
            average_rebounds, average_blocks, average_turnovers, average_efficiency, average_fast_break_pts,
            average_points_off_turnovers, average_second_chance_pts, average_minutes, average_points]
    

    


In [302]:
##### STUCK HERE HELP ###############
kansas_id = NCAA_Seasons_pd.loc[NCAA_Seasons_pd['team_name'] == 'Kansas']
season_year = NCAA_Seasons_pd.loc[NCAA_Seasons_pd['season'] == 2017]
getSeasonData(kansas_id, season_year)

NotImplementedError: iLocation based boolean indexing on an integer type is not available

In [263]:
def compareTwoTeams(team_name_1, team_name_2, year):
    team_1 = getSeasonData(team_name_1, year)
    team_2 = getSeasonData(team_name_2, year)
    diff = [a - b for a, b in zip(team_1, team_2)]
    return diff

In [264]:
kentucky_id = NCAA_Seasons_pd[NCAA_Seasons_pd['team_name'] == 'Kentucky'].values[1][2]

getSeasonData(kentucky_id, '2017')

/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/ops.py:1164: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: invalid type comparison

In [283]:
kansas_id = NCAA_Seasons_pd[NCAA_Seasons_pd['team_name'] == 'Kansas'].values[0][0]

compareTwoTeams(kansas_id, kentucky_id, 2017)

/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/ops.py:1164: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: invalid type comparison

In [266]:
def createSeasonDict(season):
    seasonDictionary = collections.defaultdict(list)
    for team in teamList:
        team_name = NCAA_Seasons_pd[NCAA_Seasons_pd['team_name'] == team].values[0][0]
        team_vector = getSeasonData(team_name, season)
        seasonDictionary[team_name] = team_vector
    return seasonDictionary

In [267]:
##### This doesn't work yet
def createTrainingSet(seasons):
    totalNumGames = 0
    for season in seasons:
        season = NCAA_Seasons_pd[NCAA_Seasons_pd['season'] == season].values[0][0]
    numFeatures = len(getSeasonData(Kansas, 2017))
    xTrain = np.zeros(( totalNumGames, numFeatures + 1))
    yTrain = np.zeros(( totalNumGames))
    indexCounter = 0
    
    xTrain[indexCounter:numGamesInSeason+indexCounter] = xTrainSeason
    yTrain[indexCounter:numGamesInSeason+indexCounter] = yTrainSeason
    indexCounter += numGamesInSeason
    return xTrain, yTrain

In [268]:
##### X & Y Training SETS####



In [269]:
##### MACHINE LEARNING #####



In [270]:
#Uncomment the model you wanna try


#model = tree.DecisionTreeClassifier()
#model = tree.DecisionTreeRegressor()
#model = linear_model.LogisticRegression()
#model = linear_model.BayesianRidge()
#model = linear_model.Lasso()
#model = svm.SVC()
#model = svm.SVR()
#model = linear_model.Ridge(alpha = 0.5)
#model = AdaBoostClassifier(n_estimators=100)
#model = GradientBoostingClassifier(n_estimators=100)
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5)
#model = RandomForestClassifier(n_estimators=64)
#model = KNeighborsClassifier(n_neighbors=39)
#neuralNetwork(10)
#model = VotingClassifier(estimators=[('GBR', model1), ('BR', model2), ('KNN', model3)], voting='soft')
#model = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.1)





In [271]:

def showDependency(predictions, test, stat, my_categories):
    difference = test[:,my_categories.index(stat)]
    plt.scatter(difference, predictions)
    plt.ylabel('Probability of Team 1 Win')
    plt.xlabel(stat + ' Difference (Team 1 - Team 2)')
    plt.show()
    
    

In [272]:

def showFeatureImportance(my_categories):
    fx_imp = pd.Series(model.feature_importances_, index=my_categories)
    fx_imp /= fx_imp.max()
    fx_imp.sort()
    fx_imp.plot(kind='barh')


In [273]:

categories=["Wins", "Adjusted Win %", "Strength of Schedule", "RPI", "Field Goal %"",
            "2-pt %", "3-pt %", "Free Throw %", "True Shooting &",
            "Average Rebounds", "Average Blocks", "Average Turnovers", "Average Efficiency", "Average Fast Break Points",
            "Average Points Off Turnover", "Average 2nd Chances", "Average Minutes", "Average Points"]
accuracy=[]

for i in range(1):
    X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain)
    results = model.fit(X_train, Y_train)
    preds = model.predict(X_test)

    preds[preds < .5] = 0
    preds[preds >= .5] = 1
    accuracy.append(np.mean(preds == Y_test))
    #accuracy.append(np.mean(predictions == Y_test))
    print "Finished iteration:", i
print "The accuracy is", sum(accuracy)/len(accuracy)

SyntaxError: EOL while scanning string literal (<ipython-input-273-444bcbd90f4b>, line 2)

In [274]:
#HERE's Where we make the cheddar



categories=["Wins", "Adjusted Win %", "Strength of Schedule", "RPI", "Field Goal %"",
            "2-pt %", "3-pt %", "Free Throw %", "True Shooting &",
            "Average Rebounds", "Average Blocks", "Average Turnovers", "Average Efficiency", "Average Fast Break Points",
            "Average Points Off Turnover", "Average 2nd Chances", "Average Minutes", "Average Points"

        

accuracy=[]

for i in range(1):
   X_train, X_test, Y_train, Y_test = train_test_split(xTrain, yTrain)
   results1 = model.fit(X_train, Y_train)
    preds1 = model.predict(X_test)
#    
#    results2 = model2.fit(X_train, Y_train)
#    preds2 = model2.predict(X_test)
    
#    results3 = model3.fit(X_train, Y_train)
#    preds3 = model3.predict(X_test)

#    results4 = model4.fit(X_train, Y_train)
#    preds4 = model4.predict(X_test)
    
#    results5 = model5.fit(X_train, Y_train)
#    preds5 = model5.predict(X_test)
    
#    preds = (preds1 + preds2 + preds3 + preds4 + preds5)/5

#    preds[preds < .5] = 0
#    preds[preds >= .5] = 1
#    accuracy.append(np.mean(preds == Y_test))
    #accuracy.append(np.mean(predictions == Y_test))
#print "The accuracy is", sum(accuracy)/len(accuracy)
#showFeatureImportance(categories)

SyntaxError: EOL while scanning string literal (<ipython-input-274-4234298bfcdb>, line 5)

In [275]:
def predictGame(team_1_vector, team_2_vector):
    diff = [a - b for a, b in zip(team_1_vector, team_2_vector)]
    return model.predict([diff]) 
    #return model.predict_proba([diff])

In [276]:

team1_name = 'North Carolina'
team2_name = 'Villanova'
team1_vector = getSeasonData(teams_pd[teams_pd['Team_Name'] == team1_name].values[0][0], 2017)
team2_vector = getSeasonData(teams_pd[teams_pd['Team_Name'] == team2_name].values[0][0], 2017)
print 'Probability that ' + team1_name + ' wins:', predictGame(team1_vector, team2_vector, 0)[0]

SyntaxError: invalid syntax (<ipython-input-276-de8c0765670c>, line 6)